In [80]:
import pandas as pd

books_file_path = 's3://cap4770-2024summer12/books_data.csv'
book_ratings_file_path = 's3://cap4770-2024summer12/Books_rating.csv'

In [81]:
def get_csv_chunk(csv_file_path, columns_to_select, chunksize):
    for chunk in pd.read_csv(csv_file_path, usecols=columns_to_select, chunksize=chunksize):
        yield chunk

### Remove unnecessary columns from books data(columns not needed for searching for a book from the frontend).

In [82]:
header = True
for chunk in get_csv_chunk(books_file_path, ['Title', 'authors', 'image'], 100000):
    chunk = chunk.rename(columns={'Title': 'title'})
    chunk = chunk.loc[chunk['title'].notnull() & chunk['authors'].notnull()] # Drop any rows with null values in the title and author columns.
    chunk.to_csv('s3://cap4770-2024summer12/filtered_books.csv', header=header, columns=['title', 'authors', 'image'], mode='a')
    header = False

### Remove unnecessary columns, drop null records from book ratings data.

In [79]:
header = True
for chunk in get_csv_chunk(book_ratings_file_path, ['User_id', 'Title', 'review/score'], 100000):
    chunk = chunk.rename(columns={'User_id': 'user_id', 'Title': 'title', 'review/score': 'rating'})
    chunk = chunk.dropna(axis=0) # Drop any rows with null values in the user_id, title, or rating columns.
    chunk.to_csv('s3://cap4770-2024summer12/filtered_book_ratings.csv', header=header, columns=['user_id', 'title', 'rating'], mode='a')
    header = False